## Part 1: Basic exploratory analysis

In [16]:
import numpy as np
import pandas as pd

In [18]:
# load data file and convert to a pandas dataframe

airfare_df = pd.read_csv('Airfares_unprocessed.csv')

In [20]:
# print first five rows
print(airfare_df.head())

  S_CODE                  S_CITY E_CODE                  E_CITY  COUPON  NEW  \
0      *  Dallas/Fort Worth   TX      *  Amarillo            TX    1.00    3   
1      *  Atlanta             GA      *  Baltimore/Wash Intl MD    1.06    3   
2      *  Boston              MA      *  Baltimore/Wash Intl MD    1.06    3   
3    ORD  Chicago             IL      *  Baltimore/Wash Intl MD    1.06    3   
4    MDW  Chicago             IL      *  Baltimore/Wash Intl MD    1.06    3   

  VACATION   SW  S_INCOME  E_INCOME    S_POP      E_POP        SLOT  GATE  \
0       No  Yes   28637.0   21112.0  3036732   205711.0        Free  Free   
1       No   No   26993.0   29838.0  3532657  7145897.0        Free  Free   
2       No   No   30124.0   29838.0  5787293  7145897.0        Free  Free   
3       No  Yes   29260.0   29838.0  7830332  7145897.0  Controlled  Free   
4       No  Yes   29260.0   29838.0  7830332  7145897.0        Free  Free   

   DISTANCE    PAX    FARE  
0       312   7864   64.11 

In [22]:
# print data types of each column

print(airfare_df.dtypes)

S_CODE       object
S_CITY       object
E_CODE       object
E_CITY       object
COUPON      float64
NEW           int64
VACATION     object
SW           object
S_INCOME    float64
E_INCOME    float64
S_POP         int64
E_POP       float64
SLOT         object
GATE         object
DISTANCE      int64
PAX           int64
FARE        float64
dtype: object


In [24]:
# print "int" type columns

print(airfare_df.select_dtypes(include=['int']).columns)

Index(['NEW', 'S_POP', 'DISTANCE', 'PAX'], dtype='object')


In [26]:
# print the length of the dataframe

print(airfare_df.shape)
print(airfare_df.shape[0])

(638, 17)
638


In [28]:
# check for null values

null_values = airfare_df.isna().sum()
column_values = null_values[null_values>0]

print(column_values)


S_INCOME    6
E_INCOME    2
E_POP       5
dtype: int64


In [30]:
# analyze "S_CITY"

airfare_df['S_CITY'].value_counts()


S_CITY
Chicago             IL    90
New York/Newark     NY    88
Atlanta             GA    41
Dallas/Fort Worth   TX    36
Los Angeles         CA    33
Boston              MA    31
Las Vegas           NV    26
Detroit             MI    22
Denver              CO    22
Houston             TX    20
Orlando             FL    15
Phoenix             AZ    15
Seattle/Tacoma      WA    12
Minneapolis/St Paul MN    12
Miami               FL    11
Fort Lauderdale     FL    11
Kansas City         MO    11
Cleveland           OH    10
Austin              TX    10
Columbus            OH    10
Albuquerque         NM     9
San Diego           CA     9
San Francisco       CA     8
Burbank             CA     7
New Orleans         LA     7
Oakland             CA     6
Philadelphia/Camden PA     6
Cincinnati          OH     6
Washington          DC     6
Hartford            CT     5
Fort Meyers         FL     4
St Louis            MO     4
Tampa               FL     4
Greenville/Sprtnbg  SC     3
Jackson

In [32]:
# analyze "E_CITY"

airfare_df['E_CITY'].value_counts()

E_CITY
New York/Newark     NY    75
Washington          DC    54
Phoenix             AZ    25
Baltimore/Wash Intl MD    23
Orlando             FL    23
                          ..
Boston              MA     1
Memphis             TN     1
El Paso             TX     1
Cincinnati          OH     1
Amarillo            TX     1
Name: count, Length: 68, dtype: int64

In [34]:
# analyze "SW"

airfare_df['SW'].value_counts()

SW
No     444
Yes    194
Name: count, dtype: int64

In [36]:
# Most frequntly appearing start city 

print(airfare_df["S_CITY"].value_counts().idxmax())

Chicago             IL


In [38]:
# Most frequntly appearing end city 

print(airfare_df["E_CITY"].value_counts().idxmax())

New York/Newark     NY


In [40]:
# Number of routes served by Southwest Airlines

print(airfare_df["SW"].value_counts().get("Yes", 0))

194


## Part 2: Data preprocessing

### Part 2.1: Select predictors

In [44]:
# drop S_CODE

airfare_df = airfare_df.drop('S_CODE', axis =1)

print(airfare_df.columns)

Index(['S_CITY', 'E_CODE', 'E_CITY', 'COUPON', 'NEW', 'VACATION', 'SW',
       'S_INCOME', 'E_INCOME', 'S_POP', 'E_POP', 'SLOT', 'GATE', 'DISTANCE',
       'PAX', 'FARE'],
      dtype='object')


In [46]:
# drop any other variables you see fit

airfare_df = airfare_df.drop('E_CODE', axis =1)

print(airfare_df.columns)

Index(['S_CITY', 'E_CITY', 'COUPON', 'NEW', 'VACATION', 'SW', 'S_INCOME',
       'E_INCOME', 'S_POP', 'E_POP', 'SLOT', 'GATE', 'DISTANCE', 'PAX',
       'FARE'],
      dtype='object')


### Part 2.2: Deal with NA values

In [49]:
# populate NA values in S_INCOME with the column mean

airfare_df["S_INCOME"] = airfare_df["S_INCOME"].fillna(airfare_df["S_INCOME"].mean())

In [51]:
# drop all rows from the DataFrame in which E_INCOME and/or E_POP takes an NA value

airfare_df.dropna(subset=["E_INCOME", "E_POP"], inplace=True)

In [53]:
# Checking for missing values

print(airfare_df.isnull().sum())

S_CITY      0
E_CITY      0
COUPON      0
NEW         0
VACATION    0
SW          0
S_INCOME    0
E_INCOME    0
S_POP       0
E_POP       0
SLOT        0
GATE        0
DISTANCE    0
PAX         0
FARE        0
dtype: int64


In [55]:
# Rows remaining after handling

print(airfare_df.shape[0])

631


### Part 2.3: Deal with Categorical Variables

In [58]:
# First we will see the categorical columns in dataframe

categorical_columns = airfare_df.select_dtypes(include=['object', 'category']).columns.tolist()
print(categorical_columns)

# S_CITY and E_CITY cannot be converted into binary flag variables

['S_CITY', 'E_CITY', 'VACATION', 'SW', 'SLOT', 'GATE']


In [60]:
# create binary flag variables for each categorical variable (this can be done with one line of code)

airfare_df = pd.get_dummies(airfare_df, columns=['VACATION', 'SW', 'SLOT', 'GATE'], drop_first=True)

In [62]:
# Columns after the encoding

print(airfare_df.columns)
print("No. of columns:", airfare_df.shape[1])

Index(['S_CITY', 'E_CITY', 'COUPON', 'NEW', 'S_INCOME', 'E_INCOME', 'S_POP',
       'E_POP', 'DISTANCE', 'PAX', 'FARE', 'VACATION_Yes', 'SW_Yes',
       'SLOT_Free', 'GATE_Free'],
      dtype='object')
No. of columns: 15


### Part 2.4 Normalize data

In [65]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler



In [66]:
# we can take the functionality from sklearn and save it under our own name to make things simpler

min_max_scaler = preprocessing.MinMaxScaler()

In [67]:
# and some more code to get you started

columns_to_normalize = ["S_INCOME","E_INCOME","S_POP","E_POP","DISTANCE","PAX","FARE"]
data_to_normalize = airfare_df[columns_to_normalize]
data_to_normalize.head()

scaler = MinMaxScaler()
airfare_df[columns_to_normalize] = scaler.fit_transform(airfare_df[columns_to_normalize])
airfare_df.head()

,S_CITY,E_CITY,COUPON,NEW,S_INCOME,E_INCOME,S_POP,E_POP,DISTANCE,PAX,FARE,VACATION_Yes,SW_Yes,SLOT_Free,GATE_Free
0,Dallas/Fort Worth TX,Amarillo TX,1.00,3,0.579730,0.268946,0.333128,0.010506,0.074717,0.087860,0.060186,False,True,True,True
1,Atlanta GA,Baltimore/Wash Intl MD,1.06,3,0.511832,0.629331,0.388071,0.786437,0.174340,0.101066,0.367126,False,False,True,True
2,Boston MA,Baltimore/Wash Intl MD,1.06,3,0.641143,0.629331,0.637858,0.786437,0.094340,0.068354,0.459714,False,False,True,True
3,Chicago IL,Baltimore/Wash Intl MD,1.06,3,0.605460,0.629331,0.864202,0.786437,0.187925,0.326573,0.119594,False,True,False,True
4,Chicago IL,Baltimore/Wash Intl MD,1.06,3,0.605460,0.629331,0.864202,0.786437,0.187925,0.326573,0.119594,False,True,True,True


In [83]:
# min values after normalization

print(airfare_df.min())

S_CITY          Albuquerque         NM
E_CITY          Amarillo            TX
COUPON                             1.0
NEW                                  0
S_INCOME                           0.0
E_INCOME                           0.0
S_POP                              0.0
E_POP                              0.0
DISTANCE                           0.0
PAX                                0.0
FARE                               0.0
VACATION_Yes                     False
SW_Yes                           False
SLOT_Free                        False
GATE_Free                        False
dtype: object


In [85]:
# max values after normalization

print(airfare_df.max())

S_CITY          Washington          DC
E_CITY          West Palm Beach     FL
COUPON                            1.94
NEW                                  3
S_INCOME                           1.0
E_INCOME                           1.0
S_POP                              1.0
E_POP                              1.0
DISTANCE                           1.0
PAX                                1.0
FARE                               1.0
VACATION_Yes                      True
SW_Yes                            True
SLOT_Free                         True
GATE_Free                         True
dtype: object
